<center>

# **Análisis de las jugadas 2**

</center>

___

In [ ]:
# Instalar las librerias para el analisis de las partidas
! pip install pandas

In [1]:
import pandas as pd
import os

#### Se importa el archivo y se crea el dataFrame

In [2]:
path_archivo = os.path.join(os.path.realpath('..'), '..', 'registros', 'datos_de_jugadas.csv')
try:
    data_set = pd.read_csv(path_archivo, encoding='utf-8')
except FileNotFoundError:
    print('No se encontró el dataset mymoviedb.csv')

In [3]:
data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   timestamp        244 non-null    float64
 1   id               244 non-null    object 
 2   evento           244 non-null    object 
 3   nombre_jugador   244 non-null    object 
 4   edad_jugador     244 non-null    int64  
 5   genero_jugador   244 non-null    object 
 6   estado           192 non-null    object 
 7   texto_ingresado  153 non-null    object 
 8   respuesta        186 non-null    object 
           244 non-null    object 
dtypes: float64(1), int64(1), object(8)
memory usage: 19.2+ KB


# Estadísticas

### Mejor Tiempo de Respuesta Correcta

* Tomo las filas que me sirven para hacer los cálculos.

In [18]:
jugadas = data_set[(data_set["evento"] == "intento") | (data_set["evento"] == "inicio_partida")]

* Duplico el dataFrame, uno de sin la primer fila y el otro sin la última. Lugo resto para obtener la duración de la respuesta.

In [19]:
last = jugadas.index[len(jugadas.index) - 1]
jugadas_resta = jugadas.drop(last).reset_index()
jugadas = jugadas.drop(0).reset_index()

jugadas['timestamp'] = jugadas['timestamp'] - jugadas_resta['timestamp']

* Busco el que tenga menor tiempo y acomodo para mostrar.

In [20]:
jugadas = jugadas[(jugadas["evento"] == "intento") & (jugadas["estado"] == "ok")]
tiempo_min = jugadas['timestamp'].min()

jugadas = jugadas.rename(columns={'timestamp': 'tiempo'}).drop('index', axis='columns')
jugadas[jugadas['tiempo'] == tiempo_min]

,tiempo,id,evento,nombre_jugador,edad_jugador,genero_jugador,estado,texto_ingresado,respuesta,nivel
142,1.112036,eea14e95-9cbb-4e8d-99b4-86bc8773340b,intento,Trinity,55,Femenino,ok,Adam Hloušek,Adam Hloušek,Personalizado


### Promedio de tiempo de respuestas exitosas

* Tomo las filas que me sirven para hacer los cálculos.

In [63]:
jugadas = data_set[(data_set["evento"] == "intento") | (data_set["evento"] == "inicio_partida")]

* Duplico el dataFrame, uno de sin la primer fila y el otro sin la última. Lugo resto para obtener la duración de la respuesta.

In [64]:
last = jugadas.index[len(jugadas.index) - 1]
jugadas_resta = jugadas.drop(last).reset_index()
jugadas = jugadas.drop(0).reset_index()

jugadas['timestamp'] = jugadas['timestamp'] - jugadas_resta['timestamp']

* Filtro el dataFrame por respuesta correcta y obtengo el promedio de tiempo. Luego adapto para mostrar.

In [65]:
jugadas = jugadas[jugadas["estado"] == "ok"]
tiempo = jugadas['timestamp'].sum() / len(jugadas)

jugadas = jugadas.loc[ :0, ['timestamp']].rename(columns={'timestamp': 'promedio de tiempo'})
jugadas['promedio de tiempo'] = tiempo
jugadas

,promedio de tiempo
0,4.627463


### Promedio de tiempo de partida

* Hago dos variables, uno con el inicio de partida y otro con el fin.

In [ ]:
partida = data_set[data_set["evento"] == "inicio_partida"].reset_index()
partida_fin = data_set[data_set["evento"] == "fin"].reset_index()

* Obtengo el promedio del tiempo de partida.

In [ ]:
partida['timestamp'] = partida_fin['timestamp'] - partida['timestamp']
tiempo = partida['timestamp'].sum() / len(partida)

* Acomodo para presentar el resultado.

In [78]:
partida = partida.loc[ :0, ['timestamp']].rename(columns={'timestamp': 'promedio de tiempo'})
partida['promedio de tiempo'] = tiempo
partida

,promedio de tiempo
0,31.513179


### Porcentaje de aciertos por usuario

* Filtro el dataFrame por las filas que necesito y obtengo cuantas veces aparece el mismo usuario.

In [36]:
jugadas = data_set[data_set["evento"] == "intento"]
jugadores = jugadas["nombre_jugador"].value_counts()

* Genero el dataFrame para los resultados.

In [37]:
res = pd.DataFrame({'jugador' : [], 'aciertos': [], 'errores': [], 'porcentaje de aciertos' : []})

* Para cada usuario guardo en los resultados la cantidad de aciertos y errores, además del porcentaje de acierto.

In [38]:
for x in jugadores.keys():
    jugador = jugadas[jugadas['nombre_jugador'] == x]['estado'].value_counts()
    porcentaje = int(jugador['ok'] / jugadores[x] * 100)
    fila = pd.DataFrame({'jugador' : [x], 'aciertos': [jugador['ok']], 'errores': [jugador['error']], 
            'porcentaje de aciertos' : [f'{porcentaje}%']})
    res = pd.concat([res, fila], ignore_index=True)
res = res.astype({'aciertos': 'int64', 'errores': 'int64'})

res

,jugador,aciertos,errores,porcentaje de aciertos
0,Trinity,9,36,20%
1,NEO,8,27,22%
2,Yoda,9,24,27%
3,Yayo,10,21,32%
4,El Jose,2,7,22%


### Cantidad de tarjetas para las que el usuarie no dió respuesta (timeout).

* Obtengo la cantidad de veces que un jugador no respondió y el total de jugadores.

In [59]:
jugadas = data_set[data_set["evento"] == "timeout"]["nombre_jugador"].value_counts()
jugadores = data_set["nombre_jugador"].value_counts().keys()

* Guardo en una variable los usuarios y en otro cuantas veces no respondió.

In [60]:
nombre = []
cant = []
for x in jugadores:
    nombre.append(x)
    if x in jugadas:
        cant.append(jugadas[x])
    else:
        cant.append(0)

* Genero un dataFrame para mostrar los resultados.

In [61]:
res = pd.DataFrame({'jugador' : nombre, 'cantidad de tarjetas sin respuesta': cant})
res

,jugador,cantidad de tarjetas sin respuesta
0,Trinity,2
1,NEO,3
2,Yayo,5
3,Yoda,0
4,El Jose,0


### Cantidad de tarjetas en las que el usuarie dió una respuesta errónea.

* Obtengo la cantidad de veces que un jugador respondió mal y el total de jugadores.

In [62]:
jugadas = data_set[(data_set["evento"] == "intento") & (data_set["estado"] == "error")]["nombre_jugador"].value_counts()
jugadores = data_set["nombre_jugador"].value_counts().keys()

* Guardo en una variable los usuarios y en otro cuantas veces respondió mal.

In [63]:
nombre = []
cant = []
for x in jugadores:
    nombre.append(x)
    if x in jugadas:
        cant.append(jugadas[x])
    else:
        cant.append(0)

* Genero un dataFrame para mostrar los resultados.

In [64]:
res = pd.DataFrame({'jugador' : nombre, 'cantidad de respuestas erroneas': cant})
res

,jugador,cantidad de respuestas erroneas
0,Trinity,36
1,NEO,27
2,Yayo,21
3,Yoda,24
4,El Jose,7
